# SQL100本ノック
[データサイエンス100本ノック（構造化データ加工編）](https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess)

手元にmysqlコンテナがあったので、こっちを使ってやってみる  

In [1]:
# pythonからmysqlにアクセスできるようにmysql-connector-pythonをインストール
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 48.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
import mysql.connector

In [10]:
# 接続パラメータの設定
config = {
  'host': os.environ['MYSQL_IP'],  
  'user': os.environ['MYSQL_USR'],
  'password': os.environ['MYSQL_PW'],
  'database': 'test'
}

# データベース接続
conn = mysql.connector.connect(**config)

# カーソルの作成
cursor = conn.cursor()

# SQLクエリの実行（例）
cursor.execute("SELECT * FROM test")

# データの取得
for row in cursor:
    print(row)




(1, 'yamada')


In [12]:
# SQLクエリの実行
cursor.execute("SELECT user from mysql.user")

# データの取得
for row in cursor:
    print(row)


('remote_user',)
('root',)
('mysql.infoschema',)
('mysql.session',)
('mysql.sys',)
('root',)


In [ ]:
# 接続のクローズ
cursor.close()
conn.close()